![](https://www.tagesspiegel.de/images/tesla/25816148/1-format6001.jpg)

Tesla, Inc. (formerly Tesla Motors, Inc.) is an American electric vehicle and clean energy company based in Palo Alto, California. Tesla's current products include electric cars (the Model S, Model 3, Model X, and Model Y), battery energy storage from home to grid scale (the Powerwall, Powerpack, and Megapack), solar products (solar panels and solar roof tiles) and related products and services.

# The dataset

In [ ]:
import numpy as np 
import pandas as pd 
import math
import matplotlib.pyplot as plt
from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam, SGD

In [ ]:
def to_datetime(df):
    date = datetime.strptime(df, '%d.%m.%Y')
    return date.strftime("%Y-%m-%d")

In [ ]:
df = pd.read_csv('../input/tesla-stock-price-new/Price Tesla.csv', sep=';')
df['Date'] = df['Date'].apply(lambda x: to_datetime(x))
df = df.sort_values('Date').reset_index(drop=True)
df.head()

In [ ]:
df.shape

In [ ]:
df = df.iloc[:,1].values
plt.plot(df)
plt.xlabel("Time")
plt.ylabel("Price")
plt.show()

In [ ]:
df = df.reshape(-1,1)

scaler = MinMaxScaler(feature_range = (0,1))
df = scaler.fit_transform(df)
np.max(df)

In [ ]:
# Test - Train Split
train_size = int(len(df) * 0.75) # % 75 Train
test_size = len(df) - train_size # % 25 Test
print("Train Size :",train_size,"Test Size :",test_size)

train = df[0:train_size,:]
test = df[train_size:len(df),:]

In [ ]:
time_stemp = 20

datax = []
datay = []
for i in range(len(train)-time_stemp-1):
    a = train[i:(i+time_stemp), 0]
    datax.append(a)
    datay.append(train[i + time_stemp, 0])
trainx = np.array(datax)
trainy = np.array(datay)


datax = []
datay = []
for i in range(len(test)-time_stemp-1):
    a = test[i:(i+time_stemp), 0]
    datax.append(a)
    datay.append(test[i + time_stemp, 0])
testx = np.array(datax)
testy = np.array(datay)

trainx = np.reshape(trainx, (trainx.shape[0], 1, trainx.shape[1]))
testx = np.reshape(testx, (testx.shape[0], 1,testx.shape[1]))
print(trainx.shape)
testx.shape

# LSTM

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU

modelGRU = Sequential()

modelGRU.add(GRU(units = 50, return_sequences = True, input_shape = (1, time_stemp)))
modelGRU.add(Dropout(0.2))

modelGRU.add(GRU(units = 50, return_sequences = True))
modelGRU.add(Dropout(0.2))

modelGRU.add(GRU(units = 50, return_sequences = True))
modelGRU.add(Dropout(0.2))

modelGRU.add(GRU(units = 50))
modelGRU.add(Dropout(0.2))

modelGRU.add(Dense(units = 1))
modelGRU.summary()

In [ ]:
epochs = 200

modelGRU.compile(optimizer="adam", loss="mean_squared_error")

callback = keras.callbacks.EarlyStopping(
    monitor="loss",
    min_delta=0,
    patience=0,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)
history = modelGRU.fit(trainx,trainy, epochs = epochs, batch_size = 128)

In [ ]:
epoch = np.arange(0, epochs, 10)
losses = []
for i in epoch:
    if i % 10 == 0:
        losses.append(history.history["loss"][i])
        
data = {"epoch":epoch,"loss":losses}
data = pd.DataFrame(data) 

import plotly.express as px

fig = px.line(data,x="epoch",y="loss",width = 800, height = 400)
fig.show()


In [ ]:
train_predict = modelGRU.predict(trainx)
test_predict = modelGRU.predict(testx)

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
trainy = scaler.inverse_transform([trainy])
testy = scaler.inverse_transform([testy])

train_score = math.sqrt(mean_squared_error(trainy[0], train_predict[:,0]))
print("Train Score : %2.f RMSE" % (train_score))
test_score = math.sqrt(mean_squared_error(testy[0], test_predict[:,0]))
print("Test Score : %2.f RMSE" % (test_score))

In [ ]:
train_predict_plot = np.empty_like(df)
train_predict_plot[:,:] = np.nan
train_predict_plot[time_stemp:len(train_predict)+time_stemp, :] = train_predict

test_predict_plot = np.empty_like(df)
test_predict_plot[:, :] = np.nan
test_predict_plot[len(train_predict)+(time_stemp*2)+1:len(df)-1, :] = test_predict

plt.figure(figsize=(20,10))
plt.plot(scaler.inverse_transform(df),color = "red",label = "Real")
plt.plot(train_predict_plot,label = "Train Predict",color = "yellow",alpha = 0.7)
plt.plot(test_predict_plot,label = "Test Predict",color = "green", alpha = 0.7)
plt.legend()
plt.xlabel("Time")
plt.ylabel("Price")
plt.show()

# LSTM

In [ ]:
time_stemp = 20

datax = []
datay = []
for i in range(len(train)-time_stemp-1):
    a = train[i:(i+time_stemp), 0]
    datax.append(a)
    datay.append(train[i + time_stemp, 0])
trainx = np.array(datax)
trainy = np.array(datay)


datax = []
datay = []
for i in range(len(test)-time_stemp-1):
    a = test[i:(i+time_stemp), 0]
    datax.append(a)
    datay.append(test[i + time_stemp, 0])
testx = np.array(datax)
testy = np.array(datay)

trainx = np.reshape(trainx, (trainx.shape[0], 1, trainx.shape[1])) # For Keras
testx = np.reshape(testx, (testx.shape[0], 1,testx.shape[1])) # For Keras
print(trainx.shape)
testx.shape

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU

modelLSTM = Sequential()

modelLSTM.add(LSTM(units = 50, return_sequences = True, input_shape = (1, time_stemp)))
modelLSTM.add(Dropout(0.2))

modelLSTM.add(LSTM(units = 50, return_sequences = True))
modelLSTM.add(Dropout(0.2))

modelLSTM.add(LSTM(units = 50, return_sequences = True))
modelLSTM.add(Dropout(0.2))

modelLSTM.add(LSTM(units = 50))
modelLSTM.add(Dropout(0.2))

modelLSTM.add(Dense(units = 1))
modelLSTM.summary()

In [ ]:
epochs = 200

modelLSTM.compile(optimizer="adam", loss="mean_squared_error")

callback = keras.callbacks.EarlyStopping(
    monitor="loss",
    min_delta=0,
    patience=0,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)
history = modelLSTM.fit(trainx,trainy, epochs = epochs, batch_size = 128)

In [ ]:
epoch = np.arange(0, epochs, 10)
losses = []
for i in epoch:
    if i % 10 == 0:
        losses.append(history.history["loss"][i])
        
data = {"epoch":epoch,"loss":losses}
data = pd.DataFrame(data)

# Visualize
import plotly.express as px

fig = px.line(data,x="epoch",y="loss",width = 800, height = 400)
fig.show()


In [ ]:
train_predict = modelLSTM.predict(trainx)
test_predict = modelLSTM.predict(testx)

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
trainy = scaler.inverse_transform([trainy])
testy = scaler.inverse_transform([testy])

train_score = math.sqrt(mean_squared_error(trainy[0], train_predict[:,0]))
print("Train Score : %2.f RMSE" % (train_score))
test_score = math.sqrt(mean_squared_error(testy[0], test_predict[:,0]))
print("Test Score : %2.f RMSE" % (test_score))

In [ ]:
train_predict_plot = np.empty_like(df)
train_predict_plot[:,:] = np.nan
train_predict_plot[time_stemp:len(train_predict)+time_stemp, :] = train_predict

test_predict_plot = np.empty_like(df)
test_predict_plot[:, :] = np.nan
test_predict_plot[len(train_predict)+(time_stemp*2)+1:len(df)-1, :] = test_predict

plt.figure(figsize=(20,10))
plt.plot(scaler.inverse_transform(df),color = "red",label = "Real")
plt.plot(train_predict_plot,label = "Train Predict",color = "yellow",alpha = 0.7)
plt.plot(test_predict_plot,label = "Test Predict",color = "green", alpha = 0.7)
plt.legend()
plt.xlabel("Time")
plt.ylabel("Price")
plt.show()

# Please upvote it if you find it useful, thank you